In [1]:
import psutil
from tqdm import tqdm
import pandas as pd
import torch
from torch.utils.data import DataLoader

import sys
sys.path.append('/root/class/cmu/LSMA/ls-hw2/code')
from modules import FeatureDataModule, MlpClassifier, FeatureDataset

/root/anaconda3/envs/ls-hw2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
best_model_path = '/shared/youngkim/dataset/mlp/cnn/version_2/checkpoints/epoch=30-step=371-val_acc=0.9440.ckpt'
model = MlpClassifier.load_from_checkpoint(best_model_path)

In [3]:
p2_df_path = '/shared/youngkim/ls-hw2/labels_p2/test_for_students.csv'
p2_df = pd.read_csv(p2_df_path)
feature_dir = '/shared/youngkim/ls-hw2/cnn_b4'
dataset = FeatureDataset(df=p2_df, feature_dir=feature_dir, p2=True)
dataloader = DataLoader(dataset, batch_size=512, drop_last=False, shuffle=False,
                        pin_memory=True, num_workers=len(psutil.Process().cpu_affinity()))

In [4]:
def inference(model, device, test_loader):
    model.eval()
    pred_y_list = []
    for data, _ in tqdm(test_loader, total=len(test_loader), desc='Infer', position=0, leave=True):
        data = data.to(device)
        
        with torch.no_grad():
            output = model(data)
            pred = output.argmax(dim=-1)

        pred_y_list.extend(pred.tolist())

    return pred_y_list

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

preds = inference(model.to(device), device, dataloader)

Infer: 100%|██████████| 58/58 [01:46<00:00,  1.84s/it]  


In [6]:
len(preds), len(p2_df)

(29694, 29694)

In [7]:
df = p2_df.copy()
df['Category'] = preds
prediction_path = '/shared/youngkim/ls-hw2/infer/eff_b4_test_prediction.csv'
df.to_csv(prediction_path, index=False)
print('Output file:', prediction_path)

Output file: /shared/youngkim/ls-hw2/infer/eff_b4_test_prediction.csv
